In [29]:
from apted import APTED
from apted.helpers import Tree
import pandas as pd
import numpy as np
import glob, os
from tqdm import tqdm
import itertools
from bs4 import BeautifulSoup
import bs4
#from collections import defaultdict

from collections import OrderedDict
from collections.abc import Callable

class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))



def defaultVal():
    return [[],0]

def makeTree(file):
    with open(file,"r") as f:
        S = f.read()
    S = S.strip()
    S = S.replace("\n","")
    #S = S.replace(" ","")
    S = S.replace("\t","")
    S = S.replace("\r","")
    soup = BeautifulSoup(S, "html.parser")
    return soup

def recursiveChildBfs(bs):
    root = bs
    stack = [root]
    count=0
    parrent = [None]
    while len(stack) != 0:
        node = stack.pop(0)
        pnode = parrent.pop(0)
        if node is not bs:
            if node.name!=None:
                yield node.name+"~"+str(count),pnode
            else:
                yield node.name,pnode
        if hasattr(node, 'children'):
            for child in node.children:
                stack.append(child)
                parrent.append(node.name+"~"+str(count))
        count+=1

def visit(tagdict,c,tree):
    tree+="{"
    tree+=c.split("~")[0]
    for i in tagdict[c][0]:
        tree = visit(tagdict,i,tree)
        tree+="}"
    return tree        

def generateTree(file):
    html = makeTree(file)
    tagdict = DefaultOrderedDict(defaultVal)
    for c,p in recursiveChildBfs(html):
        if c!=None:
            tagdict[p][0].append(c)
            tagdict[p][1]+=1


    tree = "{"
    for x,y in zip(list(tagdict.keys())[1::],list(tagdict.values())[1::]):
        tree+=x.split("~")[0]
        for c in y[0]:
            #tree+="{"
            #tree+=c
            tree = visit(tagdict,c,tree)
            tree+="}"
        tree+="}"
        break
    nNodes = 0
    for x in tagdict.keys():
        nNodes+=tagdict[x][1]
    return tree,nNodes


    
from collections import defaultdict
import pickle
import json

def load_stateMap(name):
    with open(name) as json_file:
        data = defaultdict(factory)
        data2 = json.load(json_file)
        data.update(data2)
    return data

def factory():
    return {"src":"","edges":[],"url":"","start":0}


import networkx as nx

def makeGraph_Qexplore(statedict):
    DG = nx.DiGraph()
    SFG = load_stateMap(statedict)
    
    count=1
    stateNamesBig = {}
    stateNamesSmall = {}
    edgemap = {}
    indexstate = ""
    indexlist = [x for x in SFG.keys() if SFG[x]["start"]==1]
    statesfile = [file.split("/")[-1].split(".html")[0] for file in glob.glob("../Q_Result/*.html")]

    for x in SFG.keys():
        if x in indexlist:
            stateNamesSmall["index"+str(count)] = x
            stateNamesBig[x] = "index"+str(count)
        else:
            stateNamesSmall["S"+str(count)] = x
            stateNamesBig[x] = "S"+str(count)
        count+=1
    sss = [x for x in set(statesfile).difference(set(SFG.keys())) if x not in ["index","temp"]]
    for x in sss:
        stateNamesSmall["S"+str(count)] = x
        stateNamesBig[x] = "S"+str(count)
        count+=1

    DG.add_nodes_from(stateNamesSmall.keys())
    count = 0
    for state in stateNamesSmall.keys():
        for edge in SFG[stateNamesSmall[state]]['edges']:
            if any( edge["state"] in s for s in statesfile):
                edgemap[edge["action"]] = count
                count+=1
        
    for state in stateNamesSmall.keys():
        edges = [(state,stateNamesBig[edge["state"]]) for edge in SFG[stateNamesSmall[state]]['edges'] if (edge["state"] in s for s in statesfile)]
        #DG.add_edges_from()
        DG.add_edges_from(edges)
    print("DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap")
    print(edgemap)
    return DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap

def makeGraph_crawljax(resultfile,baseurl):
    with open(resultfile) as json_file: 
        data = json.load(json_file)
    DG = nx.DiGraph()
    keys = [k for k in data["states"].keys() if baseurl in data["states"][k]["url"]]
    
    aa = os.listdir("./doms/")
    keys = [k for k in keys if k+".html" in aa]
    DG.add_nodes_from(keys)
    edges = [(edge["from"],edge["to"]) for edge in data["edges"] if edge["from"] in keys and edge["to"] in keys]
    DG.add_edges_from(edges)
    return DG,data

def generateSimplePaths_QExplore(statedict):
    DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap = makeGraph_Qexplore(statedict)
    pathdict = {}
    for node in tqdm(DG.nodes):
        for index in indexlist:
            paths = nx.all_simple_paths(DG, source=stateNamesBig[index], target=node)
            pathlist = []
            for path in paths:
                i=0
                #temp = []
                s = ""
                while(i<len(path)-1):
                    edar = []
                    for edge in SFG[stateNamesSmall[path[i]]]["edges"]:
                        if edge["state"]==stateNamesSmall[path[i+1]]:
                            #temp.append(path[i]+edge["action"])
                            #print("hello")
                            edar.append(path[i])
                    s+="|".join(set(edar))
                    s+="->"
                    i+=1
                temp = splitstring(s)
                pathlist.extend(temp)
        pathdict[node] = pathlist
    return pathdict
        
def generateSimplePaths_crawljax(resultfile,baseurl):
    DG,data = makeGraph_crawljax(resultfile,baseurl)
    pathdict = {}
    for node in tqdm(DG.nodes):
        paths = nx.all_simple_paths(DG, source="index", target=node)
        pathlist = []
        for path in paths:
            i=0
            temp = []
            while(i<len(path)-1):    
                for edge in data["edges"]:
                    if edge["from"]==path[i] and edge["to"]==path[i+1]:
                        temp.append(edge["id"])
                i+=1
            pathlist.append(temp)
        if len(pathlist)>100:
            if node in pathdict:
                pathdict[node].extend(pathlist[0:100])
            else:
                pathdict[node] = pathlist[0:100]
        else:
            if node in pathdict:
                pathdict[node].extend(pathlist)
            else:
                pathdict[node] = pathlist
        #pathdict[node] = pathlist
    return pathdict

from itertools import product
def splitstring(S):
    s = S.split("->")
    i=1
    result = s[0].split("|")
    while(i<len(s)-1):
        result= [x[0]+"->"+x[1] for x in product(result,s[i].split("|"))]
        i+=1
    #temp = [x.split("->") for x in result]
    return [x.split("->") for x in result]

def generateLocalSimplePaths_QExplore(statedict):
    DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap = makeGraph_Qexplore(statedict)
    pathdict = {}
    for node in tqdm(DG.nodes):
        for index in indexlist:
            if node!=stateNamesBig[index]:
                paths = nx.all_simple_paths(DG, source=stateNamesBig[index], target=node)
                pathlist = []
                for path in paths:
                    i=0
                    #temp = []
                    s = ""
                    while(i<len(path)-1):
                        edar = []
                        for edge in SFG[stateNamesSmall[path[i]]]["edges"]:
                            if edge["state"]==stateNamesSmall[path[i+1]]:
                                #temp.append(path[i]+edge["action"])
                                #print("hello")
                                edar.append(path[i]+str(edgemap[edge["action"]]))
                        s+="|".join(set(edar))
                        s+="->"
                        i+=1
                    temp = splitstring(s)
                    pathlist.extend(temp)
                if len(pathlist)>100:
                    if node in pathdict:
                        pathdict[node].extend(pathlist[0:100])
                    else:
                        pathdict[node] = pathlist[0:100]
                else:
                    if node in pathdict:
                        pathdict[node].extend(pathlist)
                    else:
                        pathdict[node] = pathlist
                #try:
                #    print(stateNamesBig[index],node,pathdict["index2"])
                #except:
                #    pass
                
    return pathdict
Qpaths = generateLocalSimplePaths_QExplore("../Q.map")
print(Qpaths)

def getpathSimilarity(path1,path2):
    A = set(path1)
    B = set(path2)
    shared = len(A.intersection(B))
    pathsim = (2*shared)/(len(A)+len(B))
    return pathsim

def calculate_Average_NavigationalDiversity(data):
    keys = [key for key in data.keys() if data[key]!=[]]
    diversity = []
    for key in tqdm(keys):
        paths = data[key]
        sim = []
        for path1,path2 in product(paths,paths):
            sim.append(getpathSimilarity(path1,path2))
        sim = np.array(sim)
        if np.sum(sim==1)==sim.shape[0]:
            diversity.append(1)
        else:
            diversity.append(1-np.min(sim))
    return np.mean(diversity)

def calculate_TestSuitSize(data):
    keys = [key for key in data.keys() if data[key]!=[]]
    Total = 0
    for key in tqdm(keys):
        paths = data[key]
        sim = []
        for path1,path2 in product(paths,paths):
            sim.append(getpathSimilarity(path1,path2))
        sim = np.array(sim)
        Total+=np.sum(sim==0)+1
    return Total

def calculateTestModelSize(data):
    edge = []
    for k in data.keys():
        for path in data[k]:
            edge.extend(path)
    return edge

def generateSimpleStatePaths_QExplore(statedict):
    DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap = makeGraph_Qexplore(statedict)
    pathdict = {}
    for node in tqdm(DG.nodes):
        pathlist = []
        for index in indexlist:
            paths = nx.all_simple_paths(DG, source=stateNamesBig[index], target=node)
            pathlist.extend([path for path in paths])
        if len(pathlist)>100:
            pathlist = pathlist[0:100]
        pathdict[node] = pathlist
    return pathdict
        
def generateSimpleStatePaths_crawljax(resultfile,baseurl):
    DG,data = makeGraph_crawljax(resultfile,baseurl)
    pathdict = {}
    for node in tqdm(DG.nodes):
        paths = nx.all_simple_paths(DG, source="index", target=node)
        AA = [path for path in paths]
        if len(AA)>100:
            pathdict[node] = AA[0:100]
        else:
            pathdict[node] = AA
    return pathdict

def calculateDomDiveristy(x,y,path):
    tree1,n1 = generateTree(x)
    tree2,n2 = generateTree(y)
    t1 = Tree.from_text(tree1)
    t2 = Tree.from_text(tree2)
    apted = APTED(t1, t2)
    ted = apted.compute_edit_distance()
    DD = ted/max(n1,n2)
    with open(path,"w") as file:
        file.write(str(DD))
        
def makeUniqueDom(backup,path):
    from shutil import copyfile,rmtree
    import time
    if os.path.exists(backup):
        #shutil.rmtree(backup)
        #os.mkdir(backup)
        pass
    else:
        os.mkdir(backup)

    doms = os.listdir(path)
    for dom in doms:
        copyfile(os.path.join(path,dom), os.path.join(backup,dom))
    
    for dom1 in tqdm(doms):
        domlist = []
        if dom1 in os.listdir(path) and dom1.split(".")[-1]=="html":
            tree1,n1 = generateTree(os.path.join(path,dom1))
            t1 = Tree.from_text(tree1)
            for dom2 in os.listdir(path):
                if dom2.split(".")[-1]=="html":
                    if dom1!=dom2:
                        tree2,n2 = generateTree(os.path.join(path,dom2))
                        t2 = Tree.from_text(tree2)
                        apted = APTED(t1, t2)
                        ted = apted.compute_edit_distance()
                        if ted<1:
                            domlist.append(dom2)
            for d in domlist:
                os.remove(os.path.join(path,d))
        time.sleep(0.5)

DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap
{23: 0, 29: 1, 24: 32, 30: 188, 38: 22, 35: 25, 26: 193, 28: 34, 107: 36, 31: 194, 119: 162, 32: 195, 123: 41, 68: 42, 56: 191, 45: 47, 37: 118, 64: 168, 60: 192, 62: 53, 42: 119, 61: 184, 34: 121, 65: 92, 57: 98, 33: 167, 63: 196, 46: 99, 67: 104, 71: 113, 112: 116, 108: 120, 49: 122, 88: 123, 72: 124, 81: 131, 96: 128, 133: 130, 66: 134, 70: 197, 36: 178, 58: 138, 25: 179, 51: 144, 59: 180, 79: 156, 41: 189, 116: 158, 128: 159, 134: 160, 48: 161, 44: 182, 55: 185, 50: 187}


  0%|          | 0/39 [00:00<?, ?it/s]

100%|██████████| 39/39 [00:00<00:00, 661.05it/s]

{'S2': [['index11'], ['index10']], 'S3': [['index11', 'S225'], ['index10', 'S225']], 'S4': [['index11', 'S225', 'S3188'], ['index11', 'S225', 'S332'], ['index10', 'S225', 'S3188'], ['index10', 'S225', 'S332']], 'S5': [['index11', 'S225', 'S3188', 'S436'], ['index11', 'S225', 'S3188', 'S4162'], ['index11', 'S225', 'S3188', 'S441'], ['index11', 'S225', 'S332', 'S436'], ['index11', 'S225', 'S332', 'S4162'], ['index11', 'S225', 'S332', 'S441'], ['index10', 'S225', 'S3188', 'S436'], ['index10', 'S225', 'S3188', 'S4162'], ['index10', 'S225', 'S3188', 'S441'], ['index10', 'S225', 'S332', 'S436'], ['index10', 'S225', 'S332', 'S4162'], ['index10', 'S225', 'S332', 'S441'], ['index11', 'S225', 'S3188', 'S442', 'S8120', 'S698'], ['index11', 'S225', 'S3188', 'S442', 'S8120', 'S6121'], ['index11', 'S225', 'S3188', 'S442', 'S8116', 'S698'], ['index11', 'S225', 'S3188', 'S442', 'S8116', 'S6121'], ['index11', 'S225', 'S3188', 'S442', 'S8192', 'S698'], ['index11', 'S225', 'S3188', 'S442', 'S8192', 'S612

# make Unique STATES

In [6]:
print(os.getcwd())
makeUniqueDom("../dombackup","../doms")

d:\repos\QExplore\Qexplore


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../doms'

In [7]:
makeUniqueDom("../Qbackup","../Q_Result")

100%|██████████| 47/47 [05:19<00:00,  6.79s/it]


# DOM DIVERSITY
Run process.py

# Navigational Functionality

In [30]:
Qpaths = generateLocalSimplePaths_QExplore("../Q.map")
print(Qpaths)

DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap
{23: 0, 29: 1, 24: 32, 30: 188, 38: 22, 35: 25, 26: 193, 28: 34, 107: 36, 31: 194, 119: 162, 32: 195, 123: 41, 68: 42, 56: 191, 45: 47, 37: 118, 64: 168, 60: 192, 62: 53, 42: 119, 61: 184, 34: 121, 65: 92, 57: 98, 33: 167, 63: 196, 46: 99, 67: 104, 71: 113, 112: 116, 108: 120, 49: 122, 88: 123, 72: 124, 81: 131, 96: 128, 133: 130, 66: 134, 70: 197, 36: 178, 58: 138, 25: 179, 51: 144, 59: 180, 79: 156, 41: 189, 116: 158, 128: 159, 134: 160, 48: 161, 44: 182, 55: 185, 50: 187}


100%|██████████| 39/39 [00:00<00:00, 749.97it/s]

{'S2': [['index11'], ['index10']], 'S3': [['index11', 'S225'], ['index10', 'S225']], 'S4': [['index11', 'S225', 'S3188'], ['index11', 'S225', 'S332'], ['index10', 'S225', 'S3188'], ['index10', 'S225', 'S332']], 'S5': [['index11', 'S225', 'S3188', 'S436'], ['index11', 'S225', 'S3188', 'S4162'], ['index11', 'S225', 'S3188', 'S441'], ['index11', 'S225', 'S332', 'S436'], ['index11', 'S225', 'S332', 'S4162'], ['index11', 'S225', 'S332', 'S441'], ['index10', 'S225', 'S3188', 'S436'], ['index10', 'S225', 'S3188', 'S4162'], ['index10', 'S225', 'S3188', 'S441'], ['index10', 'S225', 'S332', 'S436'], ['index10', 'S225', 'S332', 'S4162'], ['index10', 'S225', 'S332', 'S441'], ['index11', 'S225', 'S3188', 'S442', 'S8120', 'S698'], ['index11', 'S225', 'S3188', 'S442', 'S8120', 'S6121'], ['index11', 'S225', 'S3188', 'S442', 'S8116', 'S698'], ['index11', 'S225', 'S3188', 'S442', 'S8116', 'S6121'], ['index11', 'S225', 'S3188', 'S442', 'S8192', 'S698'], ['index11', 'S225', 'S3188', 'S442', 'S8192', 'S612

In [36]:
import json
with open("qpaths.json", "w") as outfile:  
    json.dump(Qpaths, outfile)

In [10]:
#import json
#with open("paths.json") as json_file:
#    Qpaths = json.load(json_file)

In [ ]:
cpaths = generateSimplePaths_crawljax("./result.json","www.saucedemo.com")

KeyError: 'states'

In [39]:
import json
with open("cpaths.json", "w") as outfile:  
    json.dump(cpaths, outfile)

NameError: name 'cpaths' is not defined

In [13]:
#Qdiversity = calculate_Average_NavigationalDiversity(Qpaths)
#Qdiversity

# run processpathfile

# Test model size

In [40]:
#Qpaths = generateLocalSimplePaths_QExplore("./Q.map")
len(set(calculateTestModelSize(Qpaths)))

36

In [ ]:
len(set(calculateTestModelSize(cpaths)))

13

# Test Suit Size

In [41]:
A = generateSimpleStatePaths_QExplore("../Q.map")

DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap
{23: 0, 29: 1, 24: 32, 30: 188, 38: 22, 35: 25, 26: 193, 28: 34, 107: 36, 31: 194, 119: 162, 32: 195, 123: 41, 68: 42, 56: 191, 45: 47, 37: 118, 64: 168, 60: 192, 62: 53, 42: 119, 61: 184, 34: 121, 65: 92, 57: 98, 33: 167, 63: 196, 46: 99, 67: 104, 71: 113, 112: 116, 108: 120, 49: 122, 88: 123, 72: 124, 81: 131, 96: 128, 133: 130, 66: 134, 70: 197, 36: 178, 58: 138, 25: 179, 51: 144, 59: 180, 79: 156, 41: 189, 116: 158, 128: 159, 134: 160, 48: 161, 44: 182, 55: 185, 50: 187}


100%|██████████| 39/39 [00:00<00:00, 1002.25it/s]


In [42]:
B = generateSimpleStatePaths_crawljax("../result.json","www.saucedemo.com")

FileNotFoundError: [Errno 2] No such file or directory: '../result.json'

In [43]:
calculate_TestSuitSize(A)

100%|██████████| 18/18 [00:00<00:00, 2000.14it/s]


18

In [44]:
calculate_TestSuitSize(B)

NameError: name 'B' is not defined

# Test Model Size Unique
run process.py

# HTML Coverage

In [45]:
import os
import cryptohash as chash
def traverseTree(tree,unique=set()):
    unique.add(tree.name)
    for x in tree.children:
        traverseTree(x,unique)
    return unique

In [46]:
path = "../Q_Result/"
exclude = [path+"index.html",path+"temp.html"]
states = [file for file in glob.glob(path+"*.html") if file not in exclude]
totaldomq = set()
for state in tqdm(states):
    tree,nodes = generateTree(state)
    tree = Tree.from_text(tree)
    taglist = traverseTree(tree)
    soup = makeTree(state)
    for tags in taglist:
        for tag in soup.find_all(tags):
            totaldomq.add(tag)
len(totaldomq)

100%|██████████| 21/21 [00:01<00:00, 12.91it/s]


435

In [ ]:
path = "../doms/"
exclude = [path+"temp.html"]
states = [file for file in glob.glob(path+"*.html") if file not in exclude]
totaldomc = set()
for state in tqdm(states):
    tree,nodes = generateTree(state)
    tree = Tree.from_text(tree)
    taglist = traverseTree(tree)
    soup = makeTree(state)
    for tags in taglist:
        for tag in soup.find_all(tags):
            totaldomc.add(tag)
len(totaldomc)

100%|██████████| 15/15 [00:00<00:00, 40.06it/s]


677

In [47]:
count = 0
with open("qpaths.json") as json_file:
    data = json.load(json_file)

for k in data.keys():
    count+=len(data[k])
count

1000

In [48]:
count = 0
with open("cpaths.json") as json_file:
    data = json.load(json_file)

for k in data.keys():
    count+=len(data[k])
count

JSONDecodeError: Expecting value: line 1 column 1 (char 0)